In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [3]:
!pip install tensorflow_text
import tensorflow_text as text
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.layers import *
import tensorflow_hub as hub
from keras.models import Model
from sklearn.model_selection import train_test_split
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
final=pd.read_csv('/content/drive/MyDrive/airline_sentiment_analysis.csv')

In [6]:
final.head()

,Unnamed: 0,airline_sentiment,text
0,1,positive,@VirginAmerica plus you've added commercials t...
1,3,negative,@VirginAmerica it's really aggressive to blast...
2,4,negative,@VirginAmerica and it's a really big bad thing...
3,5,negative,@VirginAmerica seriously would pay $30 a fligh...
4,6,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [8]:
final.isna().sum()

Unnamed: 0           0
airline_sentiment    0
text                 0
dtype: int64

In [9]:
final = final.drop('Unnamed: 0', axis = 1)

In [10]:
final.head()

,airline_sentiment,text
0,positive,@VirginAmerica plus you've added commercials t...
1,negative,@VirginAmerica it's really aggressive to blast...
2,negative,@VirginAmerica and it's a really big bad thing...
3,negative,@VirginAmerica seriously would pay $30 a fligh...
4,positive,"@VirginAmerica yes, nearly every time I fly VX..."


In [12]:
X = final['text']
y = final['airline_sentiment']

In [14]:
text = np.array(final['text'])
text = text.flatten()

In [15]:
for i in range(len(text)):
  text[i] = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text[i])
  text[i] = re.sub("(@[A-Za-z0-9_]+)", "", text[i])
  text[i] = re.sub("(#[A-Za-z0-9_]+)", "", text[i])


In [16]:
text = np.array(text)
X = pd.DataFrame(text, columns=['text'])

In [18]:
X.head()

,text
0,plus you've added commercials to the experien...
1,"it's really aggressive to blast obnoxious ""en..."
2,and it's a really big bad thing about it
3,seriously would pay $30 a flight for seats th...
4,"yes, nearly every time I fly VX this “ear wor..."


NN with LSTM

In [19]:
X = pd.DataFrame(text, columns=['text'])
y = final['airline_sentiment']

In [21]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [22]:
X_train_text = x_train.to_numpy()
x_train_text = np.asarray(X_train_text).astype(str)

X_test_text = x_test.to_numpy()
x_test_text = np.asarray(X_test_text).astype(str)


In [24]:
y_train_dum=pd.get_dummies(y_train)
y_test_dum = pd.get_dummies(y_test)

y_train_ = np.asarray(y_train_dum.values).astype('float32')
y_test_ = np.asarray(y_test_dum.values).astype('float32')

In [25]:
preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)]
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]
seq_length = 512
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
outputs = encoder(encoder_inputs)
pooled_output = outputs["sequence_output"]
embedding_model = tf.keras.Model(text_inputs, pooled_output)

In [24]:
from keras.layers import Layer

In [ ]:
text_input = Input(shape=(),dtype=tf.string,name='text')

encode_output = embedding_model(text_input)

#Flat=Flatten()(encode_output)

layer0 = keras.layers.CuDNNLSTM(128)(encode_output)
Flat = Flatten()(layer0)
layer1 = Dense(512, activation='relu')(Flat)

layer2 = Dense(256, activation='relu')(layer1)

layer3 = Dense(64, activation='relu')(layer2)
layer4 = Dense(64, activation='relu')(layer3)

output = Dense(2, activation='softmax')(layer4)
model = Model(inputs=[text_input], outputs=output)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text (InputLayer)           [(None,)]                 0         
                                                                 
 model (Functional)          (None, 512, 768)          109482241 
                                                                 
 cu_dnnlstm (CuDNNLSTM)      (None, 128)               459776    
                                                                 
 flatten_1 (Flatten)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 512)               66048     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                1644

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/bert_sentiment_LSTM128'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)




hist1 = model.fit(X_train_text, y_train_, 
                epochs=10, verbose=1, callbacks=model_checkpoint_callback)


Epoch 1/10
289/289 [==============================] - 421s 1s/step - loss: 0.2974 - accuracy: 0.8775
Epoch 2/10
289/289 [==============================] - 413s 1s/step - loss: 0.2508 - accuracy: 0.9008
Epoch 3/10
289/289 [==============================] - 410s 1s/step - loss: 0.2307 - accuracy: 0.9111
Epoch 4/10
289/289 [==============================] - 410s 1s/step - loss: 0.2020 - accuracy: 0.9198
Epoch 5/10
289/289 [==============================] - 410s 1s/step - loss: 0.1787 - accuracy: 0.9280
Epoch 6/10
289/289 [==============================] - 409s 1s/step - loss: 0.1552 - accuracy: 0.9381
Epoch 7/10
289/289 [==============================] - 409s 1s/step - loss: 0.1481 - accuracy: 0.9443
Epoch 8/10
289/289 [==============================] - 409s 1s/step - loss: 0.1235 - accuracy: 0.9532
Epoch 9/10
289/289 [==============================] - 409s 1s/step - loss: 0.1062 - accuracy: 0.9582
Epoch 10/10
289/289 [==============================] - 410s 1s/step - loss: 0.0846 - accura

In [ ]:
pred = model.predict(X_test_text)

73/73 [==============================] - 101s 1s/step


In [ ]:
y_pred = np.argmax(pred, axis=1)
y_test = np.argmax(y_test_, axis=1)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


0.9224772628843655


In [ ]:
preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)]
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]
seq_length = 128
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
outputs = encoder(encoder_inputs)
pooled_output = outputs["sequence_output"]
embedding_model = tf.keras.Model(text_inputs, pooled_output)

In [26]:
text_input = Input(shape=(),dtype=tf.string,name='text')

encode_output = embedding_model(text_input)

Flat=Flatten()(encode_output)

layer1 = Dense(128, activation='relu')(Flat)

output = Dense(2, activation='softmax')(layer1)
model = Model(inputs=[text_input], outputs=output)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text (InputLayer)           [(None,)]                 0         
                                                                 
 model (Functional)          (None, 128, 768)          109482241 
                                                                 
 flatten (Flatten)           (None, 98304)             0         
                                                                 
 dense (Dense)               (None, 128)               12583040  
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 122,065,539
Trainable params: 12,583,298
Non-trainable params: 109,482,241
_________________________________________________________________


In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/bert_text_nn_3'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)




hist1 = model.fit(X_train_text, y_train_, 
                epochs=2, batch_size = 128, verbose=1)


In [29]:
pred = model.predict(X_test_text)

73/73 [==============================] - 1184s 16s/step


In [30]:
y_pred = np.argmax(pred, axis=1)
y_test = np.argmax(y_test_, axis=1)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9233434387180598


In [ ]:
preprocessor = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)]
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]
seq_length = 512
bert_pack_inputs = hub.KerasLayer(preprocessor.bert_pack_inputs, arguments=dict(seq_length=seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
outputs = encoder(encoder_inputs)
pooled_output = outputs["sequence_output"]
embedding_model = tf.keras.Model(text_inputs, pooled_output)

In [ ]:
text_input = Input(shape=(),dtype=tf.string,name='text')

encode_output = embedding_model(text_input)

#Flat=Flatten()(encode_output)

layer0 = keras.layers.CuDNNLSTM(64)(encode_output)
Flat = Flatten()(layer0)
layer1 = Dense(512, activation='relu')(Flat)

layer2 = Dense(256, activation='relu')(layer1)

layer3 = Dense(64, activation='relu')(layer2)
layer4 = Dense(64, activation='relu')(layer3)

output = Dense(2, activation='softmax')(layer4)
model = Model(inputs=[text_input], outputs=output)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/bert_sentiment_LSTM64'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)




hist1 = model.fit(X_train_text, y_train_, 
                epochs=10, verbose=1, callbacks=model_checkpoint_callback)


Epoch 1/10
289/289 [==============================] - 420s 1s/step - loss: 0.2528 - accuracy: 0.9038
Epoch 2/10
289/289 [==============================] - 404s 1s/step - loss: 0.1988 - accuracy: 0.9247
Epoch 3/10
289/289 [==============================] - 404s 1s/step - loss: 0.1831 - accuracy: 0.9295
Epoch 4/10
289/289 [==============================] - 405s 1s/step - loss: 0.1727 - accuracy: 0.9314
Epoch 5/10
289/289 [==============================] - 403s 1s/step - loss: 0.1556 - accuracy: 0.9400
Epoch 6/10
289/289 [==============================] - 404s 1s/step - loss: 0.1356 - accuracy: 0.9454
Epoch 7/10
289/289 [==============================] - 404s 1s/step - loss: 0.1182 - accuracy: 0.9549
Epoch 8/10
289/289 [==============================] - 405s 1s/step - loss: 0.1015 - accuracy: 0.9609
Epoch 9/10
289/289 [==============================] - 404s 1s/step - loss: 0.0840 - accuracy: 0.9665
Epoch 10/10
289/289 [==============================] - 404s 1s/step - loss: 0.0626 - accura

In [ ]:
pred = model.predict(X_test_text)

73/73 [==============================] - 100s 1s/step


In [ ]:
y_pred = np.argmax(pred, axis=1)
y_test = np.argmax(y_test_, axis=1)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


0.9146816803811174
